In [ ]:
import smtplib
import glob
import docx
import pyodbc
import os 
import pandas as pd
import openpyxl as op
import re
import numpy as np
import seaborn as sns
import sys
import time
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.encoders import encode_base64
from email import encoders
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [ ]:
#Current Month Variable for Pivot Column Name Change
current_month_text = datetime.now().strftime('%B')
current_month_text

In [ ]:
#Current Month Variable for Pivot Word Doc labels
current_month_year_text = datetime.now().strftime('%B %Y')
current_month_year_text

In [ ]:
#Use current path were import file is stored
os.chdir("/Users/Jeffrey.Lu@ibm.com/Desktop/")

In [ ]:
# When importing data, save the file to your cwd with the below formatted title
df=pd.read_csv(time.strftime('Pacing Report %m-%d-%Y'))

In [ ]:
#Removing Unnecessary Columns
df.drop(['Ad_Computed_Status', 'Month_Goal_Amount','Month_Captured_Amount',\
        'Total_Captured_Amount','Current_Month_Overdelivery_Amount','Total_Overdelivery_Amount'],\
       axis = 1, inplace = True)

In [ ]:
#Convert UD Column DType to float
df['Current_Month_Estimated_Underdelivery_Amount'] = \
df['Current_Month_Estimated_Underdelivery_Amount'].str.replace(',','').astype(float)

df['Estimated_Underdelivery_Amount'] = \
df['Estimated_Underdelivery_Amount'].str.replace(',','').astype(float)


In [ ]:
# Printing and Dropping Bad AV lines

Bad_AV_Lines = df['Ad_Name'][(df['Ad_Name'].str.contains('_AV', regex = True))&\
              (df['Current_Month_Estimated_Underdelivery_Amount']!= 0)]

if df['Ad_Name'][(df['Ad_Name'].str.contains('_AV', regex = True))&\
              (df['Current_Month_Estimated_Underdelivery_Amount']!= 0)].count() > 0:
    Bad_AV_IndexNames = Bad_AV_Lines.index
    df.drop(Bad_AV_IndexNames , inplace=True)
    print('The following lines have been dropped: ' + Bad_AV_Lines)
else: 
    print('_AV lines are good, none show underdelivery')
    

In [ ]:
#Priting and Dropping Bad MG lines

Bad_MG_Lines = df['Ad_Name'][(df['Ad_Name'].str.contains('_MG', regex = True))&\
              (df['Current_Month_Estimated_Underdelivery_Amount']!= 0)]

if df['Ad_Name'][(df['Ad_Name'].str.contains('_MG', regex = True))&\
              (df['Current_Month_Estimated_Underdelivery_Amount']!= 0)].count() > 0:
    Bad_MG_IndexNames = Bad_MG_Lines.index
    df.drop(Bad_MG_IndexNames , inplace=True)
    print('The following lines have been dropped: ' + Bad_MG_Lines)
else: 
    print('_MG lines are good, none show underdelivery')

In [ ]:
#Total amount of revenue that actualized for the current month
Completed = df['Current_Month_Estimated_Underdelivery_Amount'].where(df['Ad_Status']=='complete').fillna(0).sum()
Completed

In [ ]:
#Total amount of revenue that is curently suspended
Suspended = df['Current_Month_Estimated_Underdelivery_Amount'].where(df['Ad_Status']=='suspended').fillna(0).sum()
Suspended

In [ ]:
#Advertiser Pivot
df_pivot_advertiser= pd.pivot_table(df, index= 'Advertiser', values= ['Current_Month_Estimated_Underdelivery_Amount','Estimated_Underdelivery_Amount'], aggfunc= np.sum)
df_pivot_advertiser= df_pivot_advertiser[df_pivot_advertiser.values != [0,0]].sort_values('Current_Month_Estimated_Underdelivery_Amount', ascending = False)
df_pivot_advertiser = df_pivot_advertiser.drop_duplicates()
df_pivot_advertiser['Notes for Campaigns with $10k + Risk for the Current Month']=("")
df_pivot_advertiser.rename(columns={'Current_Month_Estimated_Underdelivery_Amount': current_month_text +' Risk'}, inplace=True)
df_pivot_advertiser.rename(columns={'Estimated_Underdelivery_Amount': 'Total Risk'}, inplace=True)


df_pivot_advertiser

In [ ]:
#Changing Pivot column title to have dynamic month label
df_pivot_advertiser[current_month_text + ' Risk']=df_pivot_advertiser[current_month_text + ' Risk'].apply(lambda x: '${:,.2f}'.format(x))

In [ ]:
#Incorporating currency formatting
df_pivot_advertiser['Total Risk']=df_pivot_advertiser['Total Risk'].apply(lambda x: '${:,.2f}'.format(x))

In [ ]:
df_pivot_advertiser

In [ ]:
##### Late Creative Section #####

In [ ]:
#Late Creative file should be saved to cwd with the below formatting structure
df2=pd.read_csv(time.strftime('Late Creative %m-%d-%Y'), skipinitialspace=True)

In [ ]:
df2.head()

In [ ]:
#Deleting UK Lines
df2= df2[~df2.Region.isin(['UK', 'United Kingdom', '-'])]

In [ ]:
#Deleting Bidopt Lines
df2= df2[~df2.Name.str.contains('bidopt')]

In [ ]:
df2.reset_index(inplace = True)

In [ ]:
#Stripping CSV whitespace in column
df2['Revenue_Impact']=df2['Revenue_Impact'].str.strip()


In [ ]:
# Converting Money series to float series
df2['Revenue_Impact'] = df2['Revenue_Impact'].replace( '[\$,)]','', regex=True ).astype(float)

In [ ]:
#Stripping CSV whitespace in column
df2['Total_Month_Dollars']=df2['Total_Month_Dollars'].str.strip()
df2['Total_Month_Dollars'] = df2['Total_Month_Dollars'].replace( '[\$,)]','', regex=True ).astype(float)

In [ ]:
df2['Total_Month_Dollars'][0]

In [ ]:
#Late Creative Pivot
late_creative_pivot = pd.pivot_table(df2, index= ['Account_Executive', 'Advertiser'],values = ['Name','Total_Month_Dollars','Revenue_Impact'],aggfunc = {'Name': pd.Series.nunique, 'Total_Month_Dollars': np.sum, 'Revenue_Impact':np.sum}, margins = True)
late_creative_pivot

In [ ]:
#Adding in currency formatting to revenue_impact column
late_creative_pivot['Revenue_Impact'] = late_creative_pivot['Revenue_Impact'].apply(lambda x: '${:,.2f}'.format(x))

In [ ]:
#Adding in subtotal formatting for the Account_Executive column
late_creative_pivot_final = df2.groupby('Account_Executive').apply(lambda sub: sub.pivot_table(
    index=['Account_Executive', 'Advertiser'],
    values=['Name','Revenue_Impact','Total_Month_Dollars'],
    aggfunc={'Name': pd.Series.nunique, 'Total_Month_Dollars': np.sum, 'Revenue_Impact':np.sum},
    margins=True,
    margins_name= 'SubTotal',
))

In [ ]:
late_creative_pivot_final

In [ ]:
late_creative_pivot_final.loc[('', 'Total', '')] = late_creative_pivot_final.sum()

In [ ]:
#Removing duplicate Account_Executive column
late_creative_pivot_final.index = late_creative_pivot_final.index.droplevel(0)

In [ ]:
late_creative_pivot_final

In [ ]:
#Formatting
late_creative_pivot_final['Total_Month_Dollars']=late_creative_pivot_final['Total_Month_Dollars'].apply(lambda x: '${:,.2f}'.format(x))

In [ ]:
late_creative_pivot_final['Revenue_Impact']=late_creative_pivot_final['Revenue_Impact'].apply(lambda x: '${:,.2f}'.format(x))

In [ ]:
late_creative_pivot_final['Name']=late_creative_pivot_final['Name'].apply(lambda x: '{0:.3g}'.format(x))

In [ ]:
#Adding in notes column 
late_creative_pivot_final['Notes']=""

In [ ]:
late_creative_pivot_final.rename(columns={'Revenue_Impact': 'Pending '+ current_month_text +" Revenue"}, inplace=True)

In [ ]:
late_creative_pivot_final.rename(columns={'Total_Month_Dollars': 'Est. Revenue Impact'}, inplace=True)


In [ ]:
late_creative_pivot_final=late_creative_pivot_final.reset_index()

In [ ]:
late_creative_pivot_final

In [ ]:
#Creating list of account executives; We need to append account executive names to their respective subtotal
list1= list(late_creative_pivot_final['Account_Executive'])
list1

In [ ]:
#Creating function that extracts all "subtotal" strings from list1
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

source = list1
index_subtotal = list_duplicates_of(source, 'SubTotal')

In [ ]:
#Concatenates subtotal string to the above account executive
for i in index_subtotal:
    list1[i]=list1[i-1]+" " +list1[i]

In [ ]:
list1

In [ ]:
late_creative_pivot_final['Account_Executive'] = list1

In [ ]:
late_creative_pivot_final

In [ ]:
#Variable currency conversions

In [ ]:
Late_Creative = df2['Revenue_Impact'].sum()
Late_Creative

In [ ]:
Late_Creative_str = '${:,.0f}'.format(Late_Creative)
Late_Creative_str

In [ ]:
##Calculating Total Underdelivery and formatting into currency

In [ ]:
Current_Month_Estimated_UD = df['Current_Month_Estimated_Underdelivery_Amount'].sum()
Current_Month_Estimated_UD

In [ ]:
Current_Month_Estimated_UD_str = '${:,.0f}'.format(Current_Month_Estimated_UD)
Current_Month_Estimated_UD_str

In [ ]:
Late_Creative_Rev_Impact=df2['Revenue_Impact'].sum()
Late_Creative_Rev_Impact

In [ ]:
Late_Creative_Rev_Impact_str = '${:,.0f}'.format(Late_Creative_Rev_Impact)
Late_Creative_Rev_Impact_str

In [ ]:
Total_Underdelivery = Current_Month_Estimated_UD + Late_Creative_Rev_Impact
Total_Underdelivery

In [ ]:
Total_Underdelivery_str = '${:,.0f}'.format(Total_Underdelivery)
Total_Underdelivery_str

In [ ]:
Active_Revenue_At_Risk=df['Current_Month_Estimated_Underdelivery_Amount'].where((df['Ad_Status']=='active')).sum()
Active_Revenue_At_Risk

In [ ]:
Active_Revenue_At_Risk_str = '${:,.0f}'.format(Active_Revenue_At_Risk)
Active_Revenue_At_Risk_str

In [ ]:
#Creating week-over-week calculations
#Step 1: Create formatted aggregate variables

In [ ]:
Not_Live = Suspended + Completed + Late_Creative
Not_Live

In [ ]:
Not_Live_str = '${:,.0f}'.format(Not_Live)
Not_Live_str

In [ ]:
Suspended_str = '${:,.0f}'.format(Suspended)
Suspended_str

In [ ]:
Completed_str = '${:,.0f}'.format(Completed)
Completed_str

In [ ]:
#Printing Last File

list_of_files = glob.iglob('/Users/Jeffrey.Lu@ibm.com/Desktop/Pacing_Reports/*.xlsx')
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

In [ ]:
#Importing files from previous Pacing Report and going into calculations summary (May need to change directory)

In [ ]:
os.chdir('/Users/Jeffrey.Lu@ibm.com/Desktop/Pacing_Reports')

In [ ]:
df3=pd.read_excel(latest_file, sheet_name ='Calculations Summary')

In [ ]:
df3.head()

In [ ]:
#Create Calculation for the Previous Pacing Report:

prev_Active_Revenue_At_Risk = df3['Underdelivery Amount'][0]

prev_Active_Not_Live = df3['Underdelivery Amount'][1]

prev_Suspended = df3['Underdelivery Amount'][2]

prev_Completed = df3['Underdelivery Amount'][3]

prev_Late_Creative = df3['Underdelivery Amount'][4]


In [ ]:
#Create a calculations sheet for the current report:

d = {'Underdelivery Breakout': ['Current: Active_Revenue_At_Risk','Current: Active_Not_Live','Current: Suspended',\
                                'Current: Completed','Current: Late Creative'],'Underdelivery Amount':\
     [Active_Revenue_At_Risk, Not_Live, Suspended, Completed,Late_Creative_Rev_Impact]}
df_calculations_sheet = pd.DataFrame(data = d)

In [ ]:
df_calculations_sheet

In [ ]:
#Percentage Change Variable Calculations
Percentage_Change_Active_Revenue_At_Risk = ((Active_Revenue_At_Risk - prev_Active_Revenue_At_Risk)-1)*100
Percentage_Change_Active_Revenue_At_Risk_str = str((int(Percentage_Change_Active_Revenue_At_Risk))) + '%'
Percentage_Change_Active_Revenue_At_Risk_str

In [ ]:
Percentage_Active_Not_Live = ((Not_Live - prev_Active_Not_Live)-1)*100
Percentage_Active_Not_Live_str = str((int(Percentage_Change_Active_Revenue_At_Risk))) + '%'
Percentage_Active_Not_Live_str

In [ ]:
#Exporting Dataframes, Pivots, and Calculation Sheets to Excel Report

writer = pd.ExcelWriter(time.strftime('Pacing_Report %m-%d-%Y.xlsx'))

df.to_excel(writer,sheet_name='Pacing')
df_pivot_advertiser.to_excel(writer,sheet_name='Advertiser Breakout')
df2.to_excel(writer,sheet_name='Late Creative')
late_creative_pivot_final.to_excel(writer,sheet_name='LC Pivot')
df_calculations_sheet.to_excel(writer, sheet_name = 'Calculations Summary')


writer.save()

In [ ]:
#Completed List to Word Format Manipulation

Completed_df = df[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']].where((df['Ad_Status']=='complete')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))

Completed_df = Completed_df.groupby(['Advertiser']).sum()

Completed_df['Current_Month_Estimated_Underdelivery_Amount'] = Completed_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

Completed_df = Completed_df.reset_index()

Completed_df['Advertiser-Current_Month_UD'] = Completed_df['Advertiser'] +"- "+ Completed_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)

Completed_df


In [ ]:
#Completed List

list_Completed = []

for i in Completed_df['Advertiser-Current_Month_UD']:
    list_Completed.append(i)

if list_Completed == []:
    print('Completed lines are at $0 UD')
else:
    print(list_Completed)

In [ ]:
#Suspended List to Word Format Manipulation

Suspended_df = df[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']].where((df['Ad_Status']=='suspended')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))

Suspended_df = Suspended_df.groupby(['Advertiser']).sum()

Suspended_df['Current_Month_Estimated_Underdelivery_Amount'] = Suspended_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

Suspended_df = Suspended_df.reset_index()

Suspended_df['Advertiser-Current_Month_UD'] = Suspended_df['Advertiser'] +"- "+ Suspended_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)

Suspended_df





In [ ]:
#Suspended List

list_Suspended = []

for i in Suspended_df['Advertiser-Current_Month_UD']:
    list_Suspended.append(i)

if list_Suspended == []:
    print('Suspended lines are at $0 UD')
else:
    print(list_Suspended)

In [ ]:
#WFX List to Word Format Manipulation
df1 = df[~df['Product_Name'].str.contains('Preroll', na = False)]\
                              [df['Ad_Type'].str.contains('STANDARD|HOUSE|AD_EXCHANGE', na = False)]\
                              .where((df['Ad_Status']=='active')&(df['Product_Category']=='WEATHRFX')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))


WFX_df = df1[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']]

WFX_df = WFX_df.groupby(['Advertiser']).sum()

WFX_df['Current_Month_Estimated_Underdelivery_Amount'] = WFX_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

WFX_df = WFX_df.reset_index()

WFX_df['Advertiser-Current_Month_UD'] = WFX_df['Advertiser'] +"- "+ WFX_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)


WFX_df

In [ ]:
#WFX List

list_WFX = []

for i in WFX_df['Advertiser-Current_Month_UD']:
    list_WFX.append(i)

if list_WFX == []:
    print('WFX lines are at $0 UD')
else:
    print(list_WFX)

In [ ]:
#AFX List to Word Format Manipulation
df2 = df[~df['Product_Name'].str.contains('Preroll', na = False)]\
                              [df['Ad_Type'].str.contains('STANDARD|HOUSE|AD_EXCHANGE', na = False)]\
                              .where((df['Ad_Status']=='active')&(df['Product_Category']=='AUDIENCEFX')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))


AFX_df = df2[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']]

AFX_df = AFX_df.groupby(['Advertiser']).sum()

AFX_df['Current_Month_Estimated_Underdelivery_Amount'] = AFX_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

AFX_df = AFX_df.reset_index()

AFX_df['Advertiser-Current_Month_UD'] = AFX_df['Advertiser'] +"- "+ AFX_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)


AFX_df

In [ ]:
#AFX List

list_AFX = []

for i in AFX_df['Advertiser-Current_Month_UD']:
    list_AFX.append(i)

if list_AFX == []:
    print('WFX lines are at $0 UD')
else:
    print(list_AFX)

In [ ]:
#JFX List to Word Format Manipulation
df3 = df[~df['Product_Name'].str.contains('Preroll', na = False)]\
                              [df['Ad_Type'].str.contains('STANDARD|HOUSE|AD_EXCHANGE', na = False)]\
                              .where((df['Ad_Status']=='active')&(df['Product_Category']=='JFX')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))


JFX_df = df3[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']]

JFX_df = JFX_df.groupby(['Advertiser']).sum()

JFX_df['Current_Month_Estimated_Underdelivery_Amount'] = JFX_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

JFX_df = JFX_df.reset_index()

JFX_df['Advertiser-Current_Month_UD'] = JFX_df['Advertiser'] +"- "+ JFX_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)


JFX_df

In [ ]:
#JFX List

list_JFX = []

for i in JFX_df['Advertiser-Current_Month_UD']:
    list_JFX.append(i)

if list_JFX == []:
    print('JFX lines are at $0 UD')
else:
    print(list_JFX)

In [ ]:
#Preroll List to Word Format Manipulation

df4 = df[df['Product_Name'].str.contains('Preroll', na = False)]\
                              [df['Ad_Type'].str.contains('STANDARD|HOUSE|AD_EXCHANGE', na = False)]\
                              .where((df['Ad_Status']=='active')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))


Preroll_df = df4[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']]

Preroll_df = Preroll_df.groupby(['Advertiser']).sum()

Preroll_df['Current_Month_Estimated_Underdelivery_Amount'] = Preroll_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

Preroll_df = Preroll_df.reset_index()

Preroll_df['Advertiser-Current_Month_UD'] = Preroll_df['Advertiser'] +"- "+ Preroll_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)


Preroll_df



In [ ]:
#Preroll List

list_Preroll = []

for i in Preroll_df['Advertiser-Current_Month_UD']:
    list_Preroll.append(i)

if list_Preroll == []:
    print('Preroll lines are at $0 UD')
else:
    print(list_Preroll)

In [ ]:
#Sponsorship List to Word Format Manipulation

df5 = df.where((df['Ad_Status']=='active')&(df['Ad_Type']=='SPONSORSHIP')&(df['Current_Month_Estimated_Underdelivery_Amount']!=0))


Spon_df = df5[['Current_Month_Estimated_Underdelivery_Amount','Advertiser']]

Spon_df = Spon_df.groupby(['Advertiser']).sum()

Spon_df['Current_Month_Estimated_Underdelivery_Amount'] = Spon_df['Current_Month_Estimated_Underdelivery_Amount'].apply(lambda x: '${:,.0f}'.format(x))

Spon_df = Spon_df.reset_index()

Spon_df['Advertiser-Current_Month_UD'] = Spon_df['Advertiser'] +"- "+ Spon_df['Current_Month_Estimated_Underdelivery_Amount'].map(str)


Spon_df

In [ ]:
#Sponsorship List

list_Spon = []

for i in Spon_df['Advertiser-Current_Month_UD']:
    list_Spon.append(i)

if list_Spon == []:
    print('Sponsorship lines are at $0 UD')
else:
    print(list_Spon)

In [ ]:
#Exporting Data to Word Doc
doc = docx.Document()

paraObject = doc.add_paragraph(current_month_year_text + " Revenue at Risk (Under-Delivery+Late Creative) " + Total_Underdelivery_str)

doc.add_paragraph("Active (Live) - " + Active_Revenue_At_Risk_str + " (" + Percentage_Change_Active_Revenue_At_Risk_str + ")")

doc.add_paragraph('         *Active under-delivering campaigns')

doc.save('Pacing Report Word.docx')

In [ ]:
#All Formatted Lines
line1 = current_month_year_text + " Revenue at Risk (Under-Delivery+Late Creative) " + Total_Underdelivery_str
line2 = "Active (Live) - " + Active_Revenue_At_Risk_str + " (" + Percentage_Change_Active_Revenue_At_Risk_str + ")"
line3 = "Not Live - " + Not_Live_str + " (" + Percentage_Active_Not_Live_str + ")"
 
#Doc Object

doc = docx.Document()

#Line 1 Formatted Bold
p = doc.add_paragraph()

runner = p.add_run(line1)
runner.bold = True

#Extra Space
doc.add_paragraph()

#Line 2 Formatted Bold
p1 = doc.add_paragraph()

runner2 = p1.add_run(line2)
runner2.bold = True

#Line 3
doc.add_paragraph('         * Active under-delivering campaigns')

#Line 4 Formatted Bold
p2 = doc.add_paragraph()

runner3 = p2.add_run(line3)
runner3.bold = True

#Line 5  
doc.add_paragraph('         * Suspended- ' + Suspended_str)

#Line 6
doc.add_paragraph('         * Completed- ' + Completed_str)

#Line 7
doc.add_paragraph('         * Late Creative- ' + Late_Creative_Rev_Impact_str)



doc.save(time.strftime('Pacing Report Word %m-%d-%Y.docx'))

In [ ]:
"""Email Section"""

In [ ]:
#Creating Email Variables
email_user = 'Insert_Email_Here@gmail.com'
email_password = 'Enter_Password_Here'
email_send_to = 'Enter_Recipient_Email_Addresses_Here'
email_subject = time.strftime('Pacing_Report Materials %m-%d-%Y')

In [ ]:
#Email Object and Details

In [ ]:
msg = MIMEMultipart()

In [ ]:
msg['From'] = email_user

In [ ]:
msg['To'] = email_send_to

In [ ]:
msg['Subject'] = email_subject

In [ ]:
#Email Body > Attach body to Message Object 

In [ ]:
body = 'Please see the attaced pacing materials below. Have a great day!'

In [ ]:
msg.attach(MIMEText(body,'plain'))

In [ ]:
#File Attachment Document variables, octet-streams and base64 encoding

In [ ]:
filename1 = time.strftime('Pacing Report Word %m-%d-%Y.docx')

In [ ]:
filename2 = time.strftime('Pacing_Report %m-%d-%Y.xlsx')

In [ ]:
attachment1 = open(filename1, 'rb')

In [ ]:
attachment2 = open(filename2, 'rb')

In [ ]:
part1 = MIMEBase('application','octet-stream')

In [ ]:
part2 = MIMEBase('application','octet-stream')

In [ ]:
part1.set_payload((attachment1).read())

In [ ]:
part2.set_payload((attachment2).read())

In [ ]:
encoders.encode_base64(part1)

In [ ]:
encoders.encode_base64(part2)

In [ ]:
part1.add_header('Content-Disposition', "attachment; filename= " + filename1)

In [ ]:
part2.add_header('Content-Disposition', "attachment; filename= " + filename2)

In [ ]:
#Converting all of the Email Message as plain text

In [ ]:
msg.attach(part1)

In [ ]:
msg.attach(part2)

In [ ]:
text = msg.as_string()

In [ ]:
#Establishing SMTP Connection and tls encryption

In [ ]:
conn = smtplib.SMTP('smtp.gmail.com',587)
type(conn)

In [ ]:
conn

In [ ]:
conn.ehlo()

In [ ]:
conn.starttls()

In [ ]:
conn.login(email_user, email_password)

In [ ]:
conn.sendmail(email_user,email_send_to, text)

In [ ]:
conn.quit()

In [ ]:
#Google Sheets API Connection for Account Executive to update Notes on campaign delivery

In [ ]:
os.chdir('/Users/Jeffrey.Lu@ibm.com/Desktop')

In [ ]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('client_credentials.json', scope)

In [ ]:
client = gspread.authorize(creds)

In [ ]:
# Opening up the Google Sheets doc and inserting new sheet

In [ ]:
sheet = client.open('Late Creative Doc')

In [ ]:
sheet.add_worksheet(title= time.strftime('Late Creative %m-%d-%Y'), rows="100", cols="20") 

In [ ]:
wks = sheet.worksheet(time.strftime('Late Creative %m-%d-%Y'))

In [ ]:
#Google Sheets doesn't have insert column command so we need the pivot data laid out as rows, thus the list of variables below

In [ ]:
late_creative_pivot_final_ae = list(late_creative_pivot_final['Account_Executive'])

In [ ]:
late_creative_pivot_final_adv= list(late_creative_pivot_final['Advertiser'])

In [ ]:
late_creative_pivot_final_name= list(late_creative_pivot_final['Name'])

In [ ]:
late_creative_pivot_final_pend = list(late_creative_pivot_final['Pending '+ current_month_text +" Revenue"])

In [ ]:
late_creative_pivot_final_imp = list(late_creative_pivot_final['Est. Revenue Impact'])

In [ ]:
late_creative_pivot_final_headers = ['Account_Executive','Advertiser','Name','Pending September Revenue', 'Est. Revenue Impact', 'Notes']

In [ ]:
late_creative_pivot_final

In [ ]:
#Figuring out corresponding transpose formula alphabet letter 

In [ ]:
late_creative_pivot_len = len(late_creative_pivot_final['Account_Executive'])

In [ ]:
alphabet_list = ['a','b','c','d','e','f','g','h','i','j','k','l',
               'm','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [ ]:
alphabet_number=late_creative_pivot_len-1

In [ ]:
alphabet_number

In [ ]:
alphabet_var = alphabet_list[alphabet_number]
alphabet_var

In [ ]:
#Writing Late Creative materials to Google Sheets

In [ ]:
row1 = late_creative_pivot_final_ae
index1 = 1
wks.insert_row(row1, index1)

In [ ]:
row2 = late_creative_pivot_final_adv
index2 = 2
wks.insert_row(row2, index2)

In [ ]:
row3 = late_creative_pivot_final_name
index3 = 3
wks.insert_row(row3, index3)

In [ ]:
row4 = late_creative_pivot_final_pend
index4 = 4
wks.insert_row(row4, index4)

In [ ]:
row5 = late_creative_pivot_final_imp
index5 = 5
wks.insert_row(row5, index5)

In [ ]:
row9 = late_creative_pivot_final_headers
index9 = 9
wks.insert_row(row9, index9)

In [ ]:
wks.update_cell(10,1, '=TRANSPOSE(A1:'+alphabet_var+'5)')

In [ ]:
"""
Cron Job Established in Terminal: 
0 11 * * 1,4 /anaconda3/bin/python /Users/Jeffrey.Lu@ibm.com/Desktop/Pacing_Automation.py

Future Improvement List
#1.) Automatic data importing process via pyodbc or selenium.
#2.) Better report formatting could be written. Write python code that will insert VBA script (PC only) into the excel/Google Sheets.

#Adjustments that need to be made for when this script is inherited
1.)Script is customized for this mac computer. If changed, we need to change current work directory for Pacing Peport retrival.
2.)1st time script will run, it will use the current excel output as the current output and the previous output until
the program is run a second time for the next report where it can then use the last outputted report as the previous report
for week-over-week calculations.
3.) Email SMPT credentials username and password will need to be changed.
4.) Cron Job script will need to be entered and the variables will need to change.
5.) Google Spreadsheet and Drive API Credentails need to change.
6.) Google API Access to Drive and Sheets will need to be granted.
"""